In [ ]:
!pip install lightfm implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Разобраться с LightFM и перебрать гиперпараметры модели

Импортируем необходимые библиотеки и модули:

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from scipy.sparse import csr_matrix, coo_matrix

from lightfm import LightFM
from lightfm.evaluation import precision_at_k

import warnings
warnings.filterwarnings('ignore')

Скачиваем файлы, необходимые для работы, это:
- product.csv
- hh_demographic.csv
- etail_train.csv (его предварительно пришлось порезать на несколько небольших файлов, так как Google Colab один большой файл не переварил)

In [ ]:
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

data_response_file_names = [f'retail_train_small_{i}.csv' for i in range(11)]
data_response = pd.concat([pd.read_csv(name) for name in data_response_file_names], ignore_index=True)

Посмотрим на первые строчки файлов, которые у нас есть:

In [ ]:
display(item_features.head(2), user_features.head(2), data_response.head(2))

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


Изменим регистр признаков и название product_id и household_key:

In [ ]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

Разделим данные на train и test, оставив в тестовой выборке последние 3 недели:

In [ ]:
data_train = data_response[data_response['week_no'] < data_response['week_no'].max() - 3]
data_test = data_response[data_response['week_no'] >= data_response['week_no'].max() - 3]

Оставим ТОП-5000 товаров, а всё товары, что не войдёт в ТОП-5000, сохраним под id=999999:

In [ ]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

Создадим из train и test разреженные матрицы и посмотрим на них:

In [ ]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', aggfunc='count',
                                  fill_value=0)
user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test,
                                  index='user_id', columns='item_id',
                                  values='quantity', aggfunc='count',
                                  fill_value=0)
test_user_item_matrix = user_item_matrix.astype(float)

display(user_item_matrix.head(2), test_user_item_matrix.head(2))

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Сохраняем словари с id:

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))
itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

test_userids = test_user_item_matrix.index.values
test_itemids = test_user_item_matrix.columns.values
test_matrix_userids = np.arange(len(test_userids))
test_matrix_itemids = np.arange(len(test_itemids))
test_id_to_itemid = dict(zip(test_matrix_itemids, test_itemids))
test_id_to_userid = dict(zip(test_matrix_userids, test_userids))
test_itemid_to_id = dict(zip(test_itemids, test_matrix_itemids))
test_userid_to_id = dict(zip(test_userids, test_matrix_userids))

Собираем фичи user и item в датафреймы и применим к ним get_dummies:

In [ ]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

Финальная подготова данных для модели, делаем:
- user_item_matrix_0_1 - матрицу взаимодействий
- веса оформляем в виде координатной матрицы
- а фичи users и items в виде разреженных матриц

In [ ]:
user_item_matrix_0_1 = (sparse_user_item > 0) * 1
sample_weight = coo_matrix(user_item_matrix)
user_features = csr_matrix(user_feat_lightfm.values).tocsr()
item_features = csr_matrix(item_feat_lightfm.values).tocsr()

csr_test_user_item_matrix = csr_matrix(test_user_item_matrix).tocsr()
user_feat_lightfm_values = csr_matrix(user_feat_lightfm.values).tocsr()
item_feat_lightfm_values = csr_matrix(item_feat_lightfm.values).tocsr()

Создадим в цикле модели, изменяя в них такие гиперпараметры:
- loss - фунцию потерь:
  - BPR(Bayesian Personalized Ranking): $loss = \sigma = p^+-p^-$
  - WARP(Weighted Approximate-Rank Pairwise): $loss = ln(\frac{K-1}{N})\cdot(p^--p^+)$
- no_components - количество компонент
- learning_rate - скорость обучения
- item_alpha - фичи для item
- user_alpha - фичи для user
- Precision@5 - значение precision_at_k

Все варианты моделей соберём в сводную таблицу:

In [ ]:
%%time
result_table = pd.DataFrame({'loss': [], 'no_components': [], 'learning_rate': [],
                             'item_alpha': [], 'user_alpha': [], 'Precision@5': []})

for no_components_ in tqdm([10, 50, 100]):
    for learning_rate_ in [0.05, 0.1, 0.5]:
        for item_alpha_ in [0.001, 0.01, 0.05]:
            for user_alpha_ in [0.001, 0.01, 0.05]:
                for loss_ in ['bpr', 'warp']:
                    model = LightFM(no_components=no_components_,
                                    loss=loss_,
                                    learning_rate=learning_rate_,
                                    item_alpha=item_alpha_, user_alpha=user_alpha_,
                                    random_state=42)
                    model.fit(user_item_matrix_0_1,
                              sample_weight=sample_weight,
                              user_features=user_features,
                              item_features=item_features,
                              epochs=15, num_threads=4,
                              verbose=False)
                    test_precision = precision_at_k(model, csr_test_user_item_matrix,
                                                  user_features=user_feat_lightfm_values,
                                                  item_features=item_feat_lightfm_values,
                                                  k=5).mean()

                    result_table.loc[len(result_table)] = [loss_, no_components_, learning_rate_,
                                                            item_alpha_, user_alpha_, test_precision]

100%|██████████| 3/3 [2:47:53<00:00, 3357.68s/it]

CPU times: user 4h 45min 6s, sys: 10.6 s, total: 4h 45min 16s
Wall time: 2h 47min 53s


Вот столько моделей мы построили и обсчитали:

In [ ]:
result_table.shape[0]

162

Посмотрим на получившуюся результирующую таблицу, отсортироваоа её по значению Precision:

In [ ]:
result_table.sort_values('Precision@5', ascending=False)

,loss,no_components,learning_rate,item_alpha,user_alpha,Precision@5
15,warp,10,0.05,0.050,0.010,0.590076
141,warp,100,0.10,0.050,0.010,0.565586
61,warp,50,0.05,0.010,0.001,0.560624
109,warp,100,0.05,0.001,0.001,0.545818
55,warp,50,0.05,0.001,0.001,0.541257
...,...,...,...,...,...,...
88,bpr,50,0.10,0.050,0.050,0.163986
146,bpr,100,0.50,0.001,0.010,0.163746
90,bpr,50,0.50,0.001,0.001,0.161905
36,bpr,10,0.50,0.001,0.001,0.145978


Мы видим, что:
- размер латентного пространства не обязательно помогает получить лучший результат, зато серьёзно влияет на время обсчёта модели, увеличивая его в разы с увеличением размера самого пространства
- Остальные параметры тоже колеблятся довольно сильно как в начале таблицы с лучшим значением Precision@5, так и в конце, поэтому сказать наверняка то, что с их увеличением или уменьшением качество ранжирования стало лучше, нельзя
- Однако вот, что действительно повлияло на результат, так это выбор функции потерь. Все ТОП-5 моделей оказались с loss WARP, самые "плохие" же - с loss BPR. Это может быть связано с тем, что функция loss WARP позволяет учитывать не только индивидуальные предпочтения того или иного пользователя (как в loss BPR), но и контекст, в котором он использует систему, максимизируя количество правильно отранжированных элементов в топе рекомендаций путём выбора случайных пар и постепенного увеличения ранга правильного элемента в каждой паре. Так что выбор подходящей под конкретную задачу и данные функции потерь крайне важен.